In [31]:
#!pip install --upgrade oct2py
import numpy as np
from oct2py import octave
import matplotlib.pyplot as plt
import os
import pandas as pd
octave.addpath('C:/dynare/6.1/matlab')  
#octave.addpath('/opt/homebrew/opt/dynare/lib/dynare/matlab')

'.;C:\\dynare\\6.1\\matlab\\dseries\\src\\mdbnomics2dseries;C:\\dynare\\6.1\\matlab\\dseries\\src\\read;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\is;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\op;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\convert;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\str;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\insert;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\file;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\from;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\get;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\print;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\variables;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\cumulate;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\struct;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\misc;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\x13;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\dplot;C:\\dynare\\6.1\\matlab\\dseries\\src\\utilities\\missing\\s

In [32]:
n_goods=3

In [33]:
#from MyPy import DyPy

In [34]:
with open('t_call.m', 'w') as file:
    file.write("stoch_simul(nograph, order=1, periods=10000, irf=100);")

In [35]:
%run make_model.ipynb
%run make_vars.ipynb
%run make_parameters_and_shocks.ipynb
%run make_steady_state.ipynb


Makes a call to Octave, which makes a call to Dynare. Then the `.mod` file is run in Dynare. Output from Dynare is displayed as it runs.

In [36]:
octave.dynare('model_1.mod')
oo = octave.pull('oo_')
mm = octave.pull('M_')

Starting Dynare (version 6.1).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 12 equation(s).
Evaluating expressions...
Computing static model derivatives (order 1).
Normalizing the static model...
Finding the optimal block decomposition of the static model...
7 block(s) found:
  6 recursive block(s) and 1 simultaneous block(s).
  the largest simultaneous block has 6 equation(s)
                                 and 6 feedback variable(s).
Computing dynamic model derivatives (order 1).
Normalizing the dynamic model...
Finding the optimal block decomposition of the dynamic model...
2 block(s) found:
  1 recursive block(s) and 1 simultaneous block(s).
  the largest simultaneous block has 6 equation(s)
                                 and 6 feedback variable(s).
Preprocessing completed.
Preprocessing time: 0h00m01s.

STEADY-STATE RESULTS:

c      		 0.186403
i      		 0.358507
k      		 1.27678
y      		 0.446873
n0     		 -0.721121
s0     		 -0.7211

In [37]:
n_goods

3

In [38]:
names1 = [attr for attr in dir(oo) if not attr.startswith('__')]

Now we want to create a class where each each attribute in oo becomes an attribute of this class. So far we take only one item from mm. This is a list of endogenous variables. Below we define the class. 

In [39]:
class from_octave:
	def __init__(self, oo_structure, mm_structure, attribute_names):
		self.oo_structure = oo_structure
		self.mm_structure = mm_structure
		self. attribute_names =  attribute_names
		self.create_attributes()
		self.make_list()
     
	def create_attributes(self): 
		for attr in self.attribute_names:
			setattr(self, attr, getattr(self.oo_structure, attr)) #this is the attributes we pull from the oo structure. We are making them attributes of the class
	
	def make_list(self): 
		self.names = list(getattr(self.mm_structure, 'endo_names')) #getting one item from the mm structure. This is the endo_names
		self.names = [item[0] for item in self.names] #pulling mm gives us a list of cell arrays. We are pulling the first item from each cell array

We see that we now can reference any of the data from `oo` with the `tt.` notation. Some examples are below. 

In [40]:
tt = from_octave(oo, mm, names1)

We will care about the irfs but these are also of type `oct2py.io.Struct`. We convert it as above. We will likley make this part of the above class later, but for now we create a new class. 

In [41]:
names2 = [attr for attr in dir(tt.irfs) if attr.endswith('shock')]
#names2 = [x for _, x in sorted(zip(tt.names, names2), key=lambda pair: pair[0][0])]

I will put more desciptors here later. However, it is getting the impulse responses as difference and levels. It is simultanosuly creating two dataframes containing all differences and levels data.  

In [42]:
#recall make_levels is created in simpler ffolder
import numpy as np
import pandas as np
class make_irfs:
	def __init__(self, dta, vars, names):
		self.data = dta
		self.vars = vars
		self.names = names
		self.create_difs()
		
  	
	def create_difs(self):
		for var in self.vars:
			setattr(self, var, pd.DataFrame())  # Initialize an empty DataFrame with the dynamic name
			list_short = [name for name in self.names if name.startswith(var)]
			for attr in list_short:
				x = getattr(self.data, attr)
				x = x.flatten()
				getattr(self, var)[attr] = x  # Assign the flattened array to the DataFrame
				new_columns = [col.replace(var + '_', '').replace('shock', '') for col in getattr(self, var).columns]
				getattr(self, var).columns = new_columns
    
    



In [43]:
varlist=['c','i','y','k','n0','s0']

In [44]:
ir = make_irfs(tt.irfs,varlist, names2)
print(ir.__dict__.keys())

dict_keys(['data', 'vars', 'names', 'c', 'i', 'y', 'k', 'n0', 's0'])


Lets get a sense of what we have. 

This will also need more documentation, but it is getting the results of dynare's simulation. It defines the individual series and also puts all in a dataframe. 

In [45]:
import numpy as np

class make_data:
	def __init__(self, dta,names):
		self.data = dta
		self.names = names
		self.make_levels()
		self.make_raw()

	def make_raw(self):
		self.df_raw = pd.DataFrame()  # Initialize an empty DataFrame
		for j,attr in enumerate(self.names):
			attr=self.names[j]
			new_attr =attr + 'r'
			raw_level = self.data[j]
			setattr(self, new_attr, raw_level)
			self.df_raw[new_attr] = raw_level
			
  		
	def make_levels(self):
		self.df = pd.DataFrame()  # Initialize an empty DataFrame
		for j,attr in enumerate(self.names):
			attr=self.names[j]
			raw_level = self.data[j]
			if attr != 'i' and 'theta' not in attr:
				current_level=np.exp(raw_level)
			else:
				current_level=raw_level
			setattr(self, attr, current_level)
			self.df[attr] = current_level  # Store in DataFrame



In [46]:
eg=make_data(tt.endo_simul,tt.names)

In [47]:
def x_find(g,tt):
    thetag='theta'+str(g)
    ag='a'+str(g)
    omegag='omega'+str(g)
    thetai=eg.df[thetag][tt]
    theta0=eg.df['theta0'][tt]
    ai=eg.df[ag][tt]
    a0=eg.df['a0'][tt]
    k=eg.df['k'][tt-1]
    s0=eg.df['s0'][tt]
    n0=eg.df['n0'][tt]
    theta0=eg.df['theta0'][tt]
    eps=params['eps']
    omegai=params[omegag]
    omega0=params['omega0']
    
    x=(k*s0/n0)**((theta0-thetai)*(1-eps))*(omegai/omega0)**eps* \
    ((a0)**(1-theta0)/(ai)**(1-thetai)*(theta0/thetai)**thetai*((1-theta0)/(1-thetai))**(1-thetai))**(1-eps)
    
    return x

In [48]:
def X_find(t):
    X=0
    for g in range(0,n_goods):
        X+=x_find(g,t)
    return X

In [49]:
def Z_find(t):
    X=0
    for g in range(0,n_goods):
        thetag='theta'+str(g)
        thetai=eg.df[thetag][t]
        X+=x_find(g,t)*thetai
    return X

In [50]:
eg.df['X'] = [X_find(t) if t > 0 else np.nan for t in range(eg.df.shape[0])]
eg.df['Z'] = [Z_find(t) if t > 0 else np.nan for t in range(eg.df.shape[0])]
eg.df['Q']=(eg.df['Z']-eg.df['theta0']*eg.df['X'])/eg.df['X']

In [51]:
def py_n(tt,g):
    
    thetag='theta'+str(g)
    thetai=eg.df[thetag][tt]
    theta0=eg.df['theta0'][tt]
    a0=eg.df['a0'][tt]
    s0=eg.df['s0'][tt]
    n0=eg.df['n0'][tt]
    k=eg.df['k'][tt-1]
    prod=(1-theta0)*a0**(1-theta0)*k**theta0*(s0/n0)**theta0/(1-thetai)
    return prod

In [52]:
def n_i(tt,g):
    
    thetag='theta'+str(g)
    thetai=eg.df[thetag][tt]
    theta0=eg.df['theta0'][tt]
    y=eg.df['y'][tt]
    c=eg.df['c'][tt]
    X=eg.df['X'][tt]
    Q=eg.df['Q'][tt]
    
    xi=x_find(g,tt)
    if g>0:
        ni=xi/X*c/y*(1-thetai)/(1-theta0-c/y*Q)
    else:
        ni=(1-theta0)*(1-c/y*(1-xi/X))/(1-theta0-c/y*Q)
    return ni

In [53]:

def within_find(tt):
    within=0
    ylag=eg.df['y'][tt-1]
    a=eg.df['a0'][tt]/eg.df['a0'][tt-1]
    for g in range(0,n_goods):
        withing=(a*py_n(tt,g)-py_n(tt-1,g))*n_i(tt-1,g)
        within+=withing
    return within/ylag

In [54]:

def static_find(tt):
    static=0
    N=0
    N_minus=0
   
    ylag=eg.df['y'][tt-1]
    
    for g in range(0,n_goods):
       
        staticg=(n_i(tt,g)-n_i(tt-1,g))*py_n(tt-1,g)
        
        
        N+=n_i(tt,g)
        N_minus+=n_i(tt-1,g)
        if g==2:
            (print('check1', N,N_minus))
        if g==0:
            (print('check2',n_i(tt,g)-eg.df['n0'][tt]))
        static+=staticg
        
    return static/ylag

In [55]:
def dynamic_find(tt):
    dynamic=0
    ylag=eg.df['y'][tt-1]

    a=eg.df['a0'][tt]/eg.df['a0'][tt-1]
    for g in range(0,n_goods):
        
        dynamicg=(a*py_n(tt,g)-py_n(tt-1,g))*(n_i(tt,g)-n_i(tt-1,g))
        dynamic+=dynamicg
    return dynamic/ylag

In [56]:

eg.df['grow'] = (eg.df['y']*eg.df['a0']/eg.df['a0'].shift(1)-eg.df['y'].shift(1))/eg.df['y'].shift(1)
eg.df.loc[0, 'grow'] = np.nan
eg.df['within'] = [within_find(t) if t > 1 and t < eg.df.shape[0]-1 else np.nan for t in range(eg.df.shape[0])]
eg.df['static'] = [static_find(t) if t > 1 and t < (eg.df.shape[0]-1) else np.nan for t in range(eg.df.shape[0])]
eg.df['dynamic'] = [dynamic_find(t) if t > 1 and t < eg.df.shape[0]-1 else np.nan for t in range(eg.df.shape[0])]
eg.df['check'] = eg.df['grow']-eg.df['within']-eg.df['static']-eg.df['dynamic']


check2 2.5288077377727536e-05
check1 1.0 1.0
check2 0.0009659099570510365
check1 1.0000000000000002 1.0
check2 0.0001914612569245966
check1 0.9999999999999998 1.0000000000000002
check2 0.0038823759159534177
check1 1.0 0.9999999999999998
check2 -0.0007394887501455116
check1 0.9999999999999999 1.0
check2 0.0009631827712380803
check1 1.0 0.9999999999999999
check2 0.0022457732081028747
check1 1.0 1.0
check2 -0.006606021783682026
check1 0.9999999999999999 1.0
check2 -0.0013067680537175996
check1 1.0 0.9999999999999999
check2 -0.0031975266977854466
check1 1.0 1.0
check2 -0.014432843980907939
check1 1.0 1.0
check2 -0.0022766066518111194
check1 0.9999999999999998 1.0
check2 -1.571800608851648e-05
check1 1.0 0.9999999999999998
check2 -0.0007839820117096497
check1 0.9999999999999999 1.0
check2 -0.0020881427301860933
check1 0.9999999999999998 0.9999999999999999
check2 -0.0003119381525621967
check1 1.0 0.9999999999999998
check2 -0.007958500275923819
check1 1.0000000000000002 1.0
check2 -0.00228710

In [57]:
eg.df

,c,i,k,y,n0,s0,theta0,theta1,theta2,a0,a1,a2,X,Z,Q,grow,within,static,dynamic,check
0,1.266408,0.485108,3.713935,1.761588,0.484085,0.576450,0.417782,0.307729,0.373189,1.051757,1.059442,1.055584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.273028,0.431068,3.776511,1.708590,0.482871,0.534484,0.390100,0.348124,0.342143,1.039592,1.017201,1.120646,3.040490,1.094427,-0.030149,-0.041304,NaN,NaN,NaN,NaN
2,1.169108,0.332832,3.730106,1.502611,0.482324,0.504823,0.359763,0.356107,0.323044,0.897433,1.085343,1.099863,2.900970,1.005043,-0.013312,-0.240814,-0.242260,-0.000087,-0.000320,0.001854
3,1.098992,0.231530,3.586054,1.342782,0.504260,0.434155,0.309071,0.393035,0.357668,0.839455,1.135322,1.098793,2.741038,0.962047,0.041908,-0.164101,-0.167397,0.000562,-0.002635,0.005369
4,1.038875,0.290295,3.518372,1.335060,0.470830,0.535184,0.358702,0.306128,0.297821,0.770931,1.227985,0.902043,2.877643,0.925265,-0.037166,-0.086911,-0.076307,0.003157,-0.005873,-0.007887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.558216,0.404490,4.014442,1.962969,0.467767,0.446606,0.322389,0.294740,0.391034,1.411090,0.995541,1.221073,3.150170,1.052866,0.011836,0.030107,0.029593,0.001193,-0.002001,0.001322
9996,1.630103,0.497519,4.126220,2.157007,0.451091,0.518608,0.370239,0.290267,0.326692,1.483623,0.948270,1.304600,3.286035,1.072707,-0.043795,0.155332,0.144591,0.002290,-0.003112,0.011562
9997,1.703961,0.538969,4.278662,2.291908,0.452304,0.522645,0.390259,0.203271,0.449240,1.542961,0.843417,1.279372,3.372579,1.122150,-0.057531,0.105038,0.110261,-0.003625,-0.021666,0.020068
9998,1.537350,0.401584,4.254468,1.939071,0.444834,0.468545,0.345836,0.287445,0.358239,1.276641,0.750236,1.165150,3.270557,1.070817,-0.018425,-0.299980,-0.282922,0.015985,-0.012808,-0.020234


In [58]:

eg.df.shape[0]

10000

In [59]:
.5/3


0.16666666666666666

In [60]:

# Function to highlight the diagonal
def highlight_diagonal(data):
	attr = 'background-color: yellow'
	# Create a DataFrame with empty strings
	df_styler = pd.DataFrame('', index=data.index, columns=data.columns)
	# Set the diagonal to the desired attribute
	np.fill_diagonal(df_styler.values, attr)
	return df_styler

# Apply the function to the DataFrame
df_cov = df_cov.style.apply(highlight_diagonal, axis=None)

# Display the styled DataFrame
df_cov

NameError: name 'df_cov' is not defined